## Package Imports

In [1]:
#Edited from model_inference_alex.ipynb
    #uses threads (instead of processes) and the callback version of pyaudio instead of blocking
    #https://www.dlology.com/blog/how-to-do-real-time-trigger-word-detection-with-keras/
    #based off of ^^ link, instead of analyzing discrete 2 second chunks, does sliding predictions each half a sec


import pyaudio
import librosa
import logging
import time
import wave
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from threading import Thread
from array import array
from scipy.io import wavfile
from queue import Queue
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import Input, layers, optimizers, backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from gsmmodem.modem import GsmModem

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/librosa/display.py:32: MatplotlibDeprecationWarning: 
The examples.directory rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2. In the future, examples will be found relative to the 'datapath' directory.
  mpl.rcParams.update(**_matplotlibrc)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/_collections_abc.py:849: MatplotlibDeprecationWarning: 
The examples.directory rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2. In the future, examples will be found relative to the 'datapath' directory.
  self[key] = value
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/_collections_abc.py:849: MatplotlibDeprecationWarning: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
  self[key] = value
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/_collections_abc.py:849: MatplotlibDeprecationWarning: 
The 

## Configuring the Logger

In [2]:
logger = logging.getLogger('debugger')
logger.setLevel(logging.DEBUG)
ch = logging.FileHandler('output.log')
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

## Variable Initializations

In [3]:
audio_format = pyaudio.paInt16
audio_rate = 44100
audio_channels = 1
audio_device_index = 0
audio_frames_per_buffer = 4410
audio_sample_duration = 2
audio_analysis_queue = Queue()
audio_analysis_time_queue = Queue()
sound_data = np.zeros(0, dtype = "int16")
audio_volume_threshold = 1000
sms_alert_queue = Queue()
inference_model_confidence_threshold = 0.95
max_audio_frame_int_value = 2 ** 15 - 1
sound_normalization_threshold = 10 ** (-1.0 / 20)
designated_alert_recipients = ["8163449956", "9176202840", "7857642331"]

localization_data_queue = Queue()
localization_time_queue = Queue()

## Loading in Augmented Labels

In [4]:
labels = np.load("/home/alexm/Datasets/gunshot_augmented_sound_labels.npy")

FileNotFoundError: [Errno 2] No such file or directory: '/home/alexm/Datasets/gunshot_augmented_sound_labels.npy'

## Binarizing Labels

In [ ]:
labels = np.array([("gun_shot" if label == 1 else "other") for label in labels])
label_binarizer = LabelBinarizer()
labels = label_binarizer.fit_transform(labels)
labels = np.hstack((labels, 1 - labels))

## Sound Post-Processing Functions

In [5]:
def normalize(sound_data):
    normalization_factor = float(sound_normalization_threshold * max_audio_frame_int_value) / max(abs(i) for i in sound_data)
    
    # Averages the volume out
    r = array('h')
    for datum in sound_data:
        r.append(int(datum * normalization_factor))
    return np.array(r, dtype = np.int16)

### Librosa Wrapper Function Definitions

In [6]:
def _stft(y, n_fft, hop_length, win_length):
    return librosa.stft(y = y, n_fft = n_fft, hop_length = hop_length, win_length = win_length)


def _istft(y, hop_length, win_length):
    return librosa.istft(y, hop_length, win_length)


def _amp_to_db(x):
    return librosa.core.logamplitude(x, ref_power = 1.0, amin = 1e-20, top_db = 80.0)  # Librosa 0.4.2 functionality
#     return librosa.core.amplitude_to_db(x, ref = 1.0, amin = 1e-20, top_db = 80.0)  # Librosa 0.6.3 functionality


def _db_to_amp(x):
    return librosa.core.perceptual_weighting(x, frequencies = 1.0)  # Librosa 0.4.2 functionality
#     return librosa.core.db_to_amplitude(x, ref = 1.0)  # Librosa 0.6.3 functionality

### Custom Noise Reduction Function Definition

In [7]:
def remove_noise(audio_clip,
                noise_clip,
                n_grad_freq = 2,
                n_grad_time = 4,
                n_fft = 2048,
                win_length = 2048,
                hop_length = 512,
                n_std_thresh = 1.5,
                prop_decrease = 1.0,
                verbose = False,
                visual = False):
    
    """ Removes noise from audio based upon a clip containing only noise

    Args:
        audio_clip (array): The first parameter.
        noise_clip (array): The second parameter.
        n_grad_freq (int): how many frequency channels to smooth over with the mask.
        n_grad_time (int): how many time channels to smooth over with the mask.
        n_fft (int): number audio of frames between STFT columns.
        win_length (int): Each frame of audio is windowed by `window()`. The window will be of length `win_length` and then padded with zeros to match `n_fft`..
        hop_length (int):number audio of frames between STFT columns.
        n_std_thresh (int): how many standard deviations louder than the mean dB of the noise (at each frequency level) to be considered signal
        prop_decrease (float): To what extent should you decrease noise (1 = all, 0 = none)
        visual (bool): Whether to plot the steps of the algorithm

    Returns:
        array: The recovered signal with noise subtracted

    """
    
    # Debugging
    if verbose:
        start = time.time()
        
    # Takes a STFT over the noise sample
    noise_stft = _stft(noise_clip, n_fft, hop_length, win_length)
    noise_stft_db = _amp_to_db(np.abs(noise_stft))  # Converts the sample units to dB
    
    # Calculates statistics over the noise sample
    mean_freq_noise = np.mean(noise_stft_db, axis = 1)
    std_freq_noise = np.std(noise_stft_db, axis = 1)
    noise_thresh = mean_freq_noise + std_freq_noise * n_std_thresh
    
    # Debugging
    if verbose:
        print("STFT on noise:", td(seconds = time.time() - start))
        start = time.time()
        
    # Takes a STFT over the signal sample
    sig_stft = _stft(audio_clip, n_fft, hop_length, win_length)
    sig_stft_db = _amp_to_db(np.abs(sig_stft))
    
    # Debugging
    if verbose:
        print("STFT on signal:", td(seconds = time.time() - start))
        start = time.time()
        
    # Calculates value to which to mask dB
    mask_gain_dB = np.min(_amp_to_db(np.abs(sig_stft)))
    
    # Debugging
    if verbose:
        print("Noise Threshold & Mask Gain in dB: ", noise_thresh, mask_gain_dB)
    
    # Creates a smoothing filter for the mask in time and frequency
    smoothing_filter = np.outer(
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_freq + 1, endpoint = False),
                np.linspace(1, 0, n_grad_freq + 2),
            ]
        )[1:-1],
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_time + 1, endpoint = False),
                np.linspace(1, 0, n_grad_time + 2),
            ]
        )[1:-1]
    )
    
    smoothing_filter = smoothing_filter / np.sum(smoothing_filter)
    
    # Calculates the threshold for each frequency/time bin
    db_thresh = np.repeat(np.reshape(noise_thresh, [1, len(mean_freq_noise)]),
                          np.shape(sig_stft_db)[1],
                          axis = 0).T
    
    # Masks segment if the signal is above the threshold
    sig_mask = sig_stft_db < db_thresh
    
    # Debugging
    if verbose:
        print("Masking:", td(seconds=time.time() - start))
        start = time.time()
        
    # Convolves the mask with a smoothing filter
    sig_mask = scipy.signal.fftconvolve(sig_mask, smoothing_filter, mode="same")
    sig_mask = sig_mask * prop_decrease
    
    # Debugging
    if verbose:
        print("Mask convolution:", td(seconds=time.time() - start))
        start = time.time()
        
    # Masks the signal
    sig_stft_db_masked = (sig_stft_db * (1 - sig_mask)
                          + np.ones(np.shape(mask_gain_dB))
                          * mask_gain_dB * sig_mask)  # Masks real
    
    sig_imag_masked = np.imag(sig_stft) * (1 - sig_mask)
    sig_stft_amp = (_db_to_amp(sig_stft_db_masked) * np.sign(sig_stft)) + (1j * sig_imag_masked)
    
    # Debugging
    if verbose:
        print("Mask application:", td(seconds=time.time() - start))
        start = time.time()
        
    # Recovers the signal
    recovered_signal = _istft(sig_stft_amp, hop_length, win_length)
    recovered_spec = _amp_to_db(
        np.abs(_stft(recovered_signal, n_fft, hop_length, win_length))
    )
    
    # Debugging
    if verbose:
        print("Signal recovery:", td(seconds=time.time() - start))
        
    # Returns noise-reduced audio sample
    return recovered_signal

### WAV File Composition Function

In [8]:
# Saves a two-second gunshot sample as a WAV file
def create_gunshot_wav_file(microphone_data, index, timestamp, number_of_audio_channels = audio_channels, sample_width = 2, frame_rate = 22050):
    wav_file = wave.open("/Users/laurenogden/Downloads/"
                            + str(index) + " ("
                            + str(timestamp) + ").wav", "wb")
    wav_file.setnchannels(number_of_audio_channels)
    wav_file.setsampwidth(sample_width)
    wav_file.setframerate(frame_rate)
    wav_file.writeframes(microphone_data.reshape(44100))
    wav_file.close()

## Loading in Noise Sample

In [9]:
noise_sample_wav = "/Users/laurenogden/Downloads/1562604212.175424.wav"
noise_sample_rate, noise_sample = wavfile.read(noise_sample_wav)
noise_clip = noise_sample[14000:18000]  # Finding a clip with just noise

## Model Construction Functions

#### ROC (AUC) metric - Uses the import "from tensorflow.keras import backend as K"

In [10]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

#### 1D Time-Series Model

In [11]:
def load_model_one(weights_file):
    # Initializing 1D Time-Series Model Parameters
    drop_out_rate = 0.1
    learning_rate = 0.001
    number_of_epochs = 100
    number_of_classes = 2
    batch_size = 32
    optimizer = optimizers.Adam(learning_rate, learning_rate / 100)
    input_shape = (44100, 1)
    input_tensor = Input(shape = input_shape)
    metrics = [auc, "accuracy"]
    
    # Reconstructing 1D Time-Series Model
    x = layers.Conv1D(16, 9, activation = "relu", padding = "same")(input_tensor)
    x = layers.Conv1D(16, 9, activation = "relu", padding = "same")(x)
    x = layers.MaxPool1D(16)(x)
    x = layers.Dropout(rate = drop_out_rate)(x)

    x = layers.Conv1D(32, 3, activation = "relu", padding = "same")(x)
    x = layers.Conv1D(32, 3, activation = "relu", padding = "same")(x)
    x = layers.MaxPool1D(4)(x)
    x = layers.Dropout(rate = drop_out_rate)(x)

    x = layers.Conv1D(32, 3, activation = "relu", padding = "same")(x)
    x = layers.Conv1D(32, 3, activation = "relu", padding = "same")(x)
    x = layers.MaxPool1D(4)(x)
    x = layers.Dropout(rate = drop_out_rate)(x)

    x = layers.Conv1D(256, 3, activation = "relu", padding = "same")(x)
    x = layers.Conv1D(256, 3, activation = "relu", padding = "same")(x)
    x = layers.GlobalMaxPool1D()(x)
    x = layers.Dropout(rate = (drop_out_rate * 2))(x) # Increasing drop-out rate here to prevent overfitting

    x = layers.Dense(64, activation = "relu")(x)
    x = layers.Dense(1028, activation = "relu")(x)
    
    # Compiling 1D Time-Series Model
    output_tensor = layers.Dense(number_of_classes, activation = "softmax")(x)
    model = tf.keras.Model(input_tensor, output_tensor)
    model.compile(optimizer = optimizer, loss = keras.losses.binary_crossentropy, metrics = metrics)
    
    # Loading 1D Time-Series Model Weights
    model.load_weights(weights_file)
    
    return model

#### 2D Spectrogram Model

In [12]:
def load_model_two(weights_file):
    # 2D Spectrogram Model Parameters
    input_shape = (128, 87, 1)
    input_tensor = Input(shape = input_shape)
    learning_rate = 0.001
    optimizer = optimizers.Adam(learning_rate, learning_rate / 100)
    filter_size = (3,3)
    maxpool_size = (3,3)
    activation = "relu"
    drop_out_rate = 0.1
    number_of_classes = 2
    metrics = [auc, "accuracy"]
    
    # Reconstructing 2D Spectrogram Model
    x = layers.Conv2D(16, filter_size, activation = activation, padding = "same")(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D(maxpool_size)(x)
    x = layers.Dropout(rate = drop_out_rate)(x)

    x = layers.Conv2D(32, filter_size, activation = activation, padding = "same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D(maxpool_size)(x)
    x = layers.Dropout(rate = drop_out_rate)(x)

    x = layers.Conv2D(64, filter_size, activation = activation, padding = "same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D(maxpool_size)(x)
    x = layers.Dropout(rate = drop_out_rate)(x)

    x = layers.Conv2D(256, filter_size, activation = activation, padding = "same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalMaxPool2D()(x)
    x = layers.Dropout(rate = (drop_out_rate * 2))(x) # Increasing drop-out rate here to prevent overfitting

    x = layers.Dense(64, activation = activation)(x)
    x = layers.Dense(1028, activation = activation)(x)
    
    # Compiling 2D Spectrogram Model
    output_tensor = layers.Dense(number_of_classes, activation = "softmax")(x)
    spec_model = tf.keras.Model(input_tensor, output_tensor)
    spec_model.compile(optimizer = optimizer, loss = keras.losses.binary_crossentropy, metrics = metrics)

    # Loading 2D Spectrogram Model Weights
    spec_model.load_weights(weights_file)
    
    return spec_model

## ---

# Multithreaded Inference: A callback thread adds 0.5 second samples of microphone data to the audio analysis queue; The main thread, an audio analysis thread, detects the presence of gunshot sounds in samples retrieved from the audio analysis queue; And an SMS alert thread dispatches groups of messages to designated recipients.

## ---

## Defining Threads

### SMS Alert Thread

In [13]:
def send_sms_alert():
    # Continuously dispatches SMS alerts to a list of designated recipients
    while True:
        sms_alert_status = sms_alert_queue.get()
        if sms_alert_status == "Gunshot Detected":
            logger.debug("ALERT: A Gunshot Has Been Detected")
    
    """
    
    # Configuring the Modem Connection
    modem_port = '/dev/ttyUSB0'
    modem_baudrate = 115200
    modem_sim_pin = None  # SIM card PIN (if any)
    
    # Establishing a Connection to the SMS Modem
    logger.debug("Initializing connection to modem...")
    modem = GsmModem(modem_port, modem_baudrate)
    modem.smsTextMode = False
    modem.connect(modem_sim_pin)
    
    # The SMS alert thread will run indefinitely
    while True:
        sms_alert_status = sms_alert_queue.get()
        if sms_alert_status == "Gunshot Detected":
            try:
                # At this point in execution, an attempt to send an SMS alert to local authorities will be made
                modem.waitForNetworkCoverage(timeout = 86400)
                message = "(Testing) ALERT: A Gunshot Has Been Detected (Testing)"
                for number in designated_alert_recipients:
                    modem.sendSms(number, message)
                logger.debug(" *** Sent out an SMS alert to all designated recipients *** ")
            except:
                logger.debug("ERROR: Unable to successfully send an SMS alert to the designated recipients.")
                pass
            finally:
                logger.debug(" ** Finished evaluating an audio sample with the model ** ")
    
    """

### Time Localization Thread

In [14]:
#get data from the time localization queue
#writes out a wav file starting when a gunshot was detected and 

def localize():
    
    #variables
    all_mic_data = []
    #index of iteration
    loop = 0
    
    #infinite loop
    while True:
        #get all the mic data from the queue and add it to the list
        while True:
            chunk = localization_data_queue.get()
            if np.array_equal(chunk, (np.zeros((44100,), dtype = np.int16))):
                logger.debug("    reached end of a gunshot group")
                break;
            else:
                all_mic_data.append(chunk) 

                
        #get beginning of mic data times
        time_at_beg_of_first_gunshot_clip = localization_time_queue.get()
        
        # processing
        np_all_mic_data = np.concatenate(all_mic_data)
        ref_all_mic_data = librosa.resample(y=np_all_mic_data, orig_sr=44100, target_sr=22050)
        ref_all_mic_data = librosa.util.normalize(ref_all_mic_data)
        
        #write out a soundfile of the entire chunk
        with sf.SoundFile("/Users/laurenogden/Downloads/" + str(loop) + "__" + time.ctime(time_at_beg_of_first_gunshot_clip) + ".wav", mode='wb', samplerate=22050, channels=1) as file:
            file.write(ref_all_mic_data)
        logger.debug("    wrote out the #" + str(loop) + "chunk, size: " + str(len(ref_all_mic_data)))

 
        #figure out the times of the gunshots in the clip
        #sort the data, figure out the threshold
        sorted_data = np.sort(ref_all_mic_data)
        threshold = sorted_data[len(sorted_data) - int(len(sorted_data)*0.001)]

        #find all values above that threshold
        above_threshold = []
        for i in range(0, len(ref_all_mic_data)):
            if ref_all_mic_data[i] > threshold:
                above_threshold.append(i)
                
        #separate out individual gunshots from that whole chunk
        distinct_shots = []
        distinct_shots.append(above_threshold[0])
        for i in range(1, len(above_threshold)):
            #if within 5ms of each other, assume from same shot
            if above_threshold[i] - above_threshold[i-1] > 0.05*22050:
                distinct_shots.append(above_threshold[i])

        #times relative to beginning of the saved clip
        logger.debug("There were " + str(len(distinct_shots)) + " distinct shots detected at " + str(time.ctime(time_at_beg_of_first_gunshot_clip)))
        for i in distinct_shots:
            logger.debug(i/22050)
            
        #times in general
        #logger.debug("The exact times of the gunshots were: ")
        #for i in distinct_shots:
            #logger.debug(time_at_beg_of_first_gunshot_clip + i/22050 - 2)
            #logger.debug(time.ctime(time_at_beg_of_first_gunshot_clip + i/22050 - 2))

            
        #clear all mic data to be ready for the next clip
        del all_mic_data[:]
        #increase index
        loop += 1
        

### Callback Thread

In [15]:
def callback(in_data, frame_count, time_info, status):
    global sound_data;
    sound_buffer = np.frombuffer(in_data, dtype = "int16")
    sound_data = np.append(sound_data, sound_buffer)
    
    #put each half a second on the queue
    if len(sound_data) >= 22050:
        audio_analysis_queue.put(sound_data)
        #put the time on a separate queue
        current_time = time.time()
        audio_analysis_time_queue.put(current_time)
        #empty out sound_data
        sound_data = np.zeros(0, dtype = "int16")
        

    return (sound_buffer, pyaudio.paContinue)

### Opening and Capturing the Microphone Audio Stream

In [16]:
pa = pyaudio.PyAudio()

stream = pa.open(format = audio_format,
                 rate = audio_rate,
                 channels = audio_channels,
                 input_device_index = audio_device_index,
                 frames_per_buffer = audio_frames_per_buffer,
                 input = True,
                 stream_callback = callback)

# Starts the callback thread
stream.start_stream()
logger.debug("--- Listening to Audio Stream ---")

### Analyzing Microphone Audio

In [17]:
# Starts the SMS alert thread
sms_alert_thread = Thread(target = send_sms_alert)
sms_alert_thread.start()

#starting the time localization thread
#time_localization_thread = Thread(target = localization)
#time_localization_thread.start()

# Loading 1D Time-Series Model
model = load_model_one("../models/gunshot_sound_model.h5")
    
# Loading 2D Spectrogram Model
#   model = load_model_two("./models/gunshot_sound_model_spectrograph_model.h5")
    
# An iterator variable for counting the number of gunshot sounds detected
gunshot_sound_counter = 1

#booleans for detection status
detected = False
detected_first = False
detected_again = False
end_of_detections = False

# get the first 2 s long clip from the queue
all_mic_data = []
for i in range(0, 4):
    all_mic_data.append(np.array(audio_analysis_queue.get(), dtype = "int16"))    
#make it into a numpy array
microphone_data = np.concatenate(all_mic_data)
# and get the start time at the beg of those 2 secs
time_of_sample_occurrence = audio_analysis_time_queue.get()

times_of_shots = []

# The main (audio analysis) thread will run indefinitely
while True:

    # Outputs the current sample's maximum frequency value
    maximum_frequency_value = max(microphone_data)
    logger.debug("The maximum frequency value of a given sample: " + str(maximum_frequency_value))
        
    # Determines whether a given sample potentially contains a gunshot
    if maximum_frequency_value >= audio_volume_threshold:
        # Post-processes the microphone data
        modified_microphone_data = librosa.resample(y = microphone_data, orig_sr = audio_rate, target_sr = 22050)
        modified_microphone_data = normalize(modified_microphone_data)
#         modified_microphone_data = remove_noise(audio_clip = modified_microphone_data, noise_clip = noise_clip)  # As a substitute for normalization
#         number_of_missing_sample_hertz = 44100 - len(modified_microphone_data)
#         if number_of_missing_sample_hertz > 0:
#             modified_microphone_data = np.array(modified_microphone_data.tolist() + [0 for i in range(number_of_missing_sample_hertz)])
        modified_microphone_data = modified_microphone_data[:44100]
        modified_microphone_data = modified_microphone_data.reshape(-1, 44100, 1)

        # Passes a given audio sample into the model for prediction
        probabilities = model.predict(modified_microphone_data)
        logger.debug("The model-predicted probability values: " + str(probabilities[0]))
        #logger.debug("Model-predicted sample class: " + label_binarizer.inverse_transform(probabilities[:, 0])[0])

        # Determines if a gunshot sound was detected by the model
        if (probabilities[0][1] >= inference_model_confidence_threshold):
            
            # Sends out an SMS alert
            sms_alert_queue.put("Gunshot Detected")

            # Makes a WAV file of the gunshot sample
            create_gunshot_wav_file(modified_microphone_data, gunshot_sound_counter, time_of_sample_occurrence)

            # Increments the counter for gunshot sound file names
            gunshot_sound_counter += 1
            
            
            
            
            #time localization in here 
            #figure out the times of the gunshots in the clip
            #sort the data, figure out the threshold
            modified_microphone_data = modified_microphone_data.reshape(44100)
            sorted_data = np.sort(modified_microphone_data)
            threshold = sorted_data[len(sorted_data) - int(len(sorted_data)*0.001)]

            #find all values above that threshold
            above_threshold = []
            for i in range(0, len(modified_microphone_data)):
                if modified_microphone_data[i] > threshold:
                    above_threshold.append(i)

            #separate out individual gunshots from that whole chunk
            distinct_shots = []
            distinct_shots.append(above_threshold[0])
            for i in range(1, len(above_threshold)):
                #if within 5ms of each other, assume from same shot
                if above_threshold[i] - above_threshold[i-1] > 0.05*22050:
                    distinct_shots.append(above_threshold[i])

            #times relative to beginning of the saved clip
            logger.debug("There were " + str(len(distinct_shots)) + " distinct shots detected at " + str(time.ctime(time_of_sample_occurrence)))
            for i in distinct_shots:
                logger.debug(i/22050)
                times_of_shots.append(time_of_sample_occurrence + i/22050)
                
                
                
            
            #update booleans w detection state 
            if not detected:
                detected = True
                detected_first = True
            else:
                detected_first = False
                detected_again = True
                    
        else:
            #update booleans w detection state
            if detected:
                end_of_detections = True
            else:
                end_of_detections = False
            detected = False
            detected_first = False
            detected_again = False
            
    else:
        #update booleans w detection state
        if detected:
            end_of_detections = True
        else:
            end_of_detections = False
        detected = False
        detected_first = False
        detected_again = False
        
     #based on booleans, put microphone data or partial mic data on the time localization queue
    if detected_first:
        localization_data_queue.put(microphone_data)
        localization_time_queue.put(time_of_sample_occurrence)
    if detected_again:
        localization_data_queue.put(microphone_data[66150:]) #just the last half second
    if end_of_detections:
        localization_data_queue.put(microphone_data[66150:]) #just the last half second
        localization_data_queue.put(np.zeros((44100,), dtype = np.int16))
        end_of_detections = False
            
            
    #get a new 2 second long sample
    #get rid of the first half a second of current microphone data
    microphone_data = microphone_data[22050:]
    #get a new half second from the queue to add onto the end
    microphone_data = np.append(microphone_data, np.array(audio_analysis_queue.get(), dtype = "int16"))
    #get a new time for the new beginning from the time queue
    time_of_sample_occurrence = audio_analysis_time_queue.get()
        
        

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


KeyboardInterrupt: 

In [25]:
times_of_shots.sort()

times_of_distinct_shots = []
times_of_distinct_shots.append(times_of_shots[0])
j = 0

for i in range(1, len(times_of_shots)):
    #print("diff from before:" times_of_shots[i] - times_of_shots[i-1])
    #print(times_of_shots[i] - times_of_distinct_shots[j])
    if times_of_shots[i] - times_of_shots[i-1] > 0.05:
        times_of_distinct_shots.append(times_of_shots[i])
        j += 1
        
        
print("# shots: " + str(len(times_of_shots)))
print("# distinct shots: " + str(len(times_of_distinct_shots)))

print("all:")
for a in times_of_shots:
    print(a)
    
print("distinct:")
for b in times_of_distinct_shots:
    print(b)


# shots: 14
# distinct shots: 5
all:
1562621802.3592997
1562621802.3810155
1562621802.4165487
1562621802.4866014
1562621802.5083172
1562621802.5438504
1562621802.5930412
1562621802.614757
1562621802.6502903
1562621802.719481
1562621802.7411969
1562621802.77673
1562621804.3456922
1562621805.2138555
distinct:
1562621802.3592997
1562621802.4866014
1562621802.719481
1562621804.3456922
1562621805.2138555


## Testing A Model with Sample Audio (Optional)

In [ ]:
# model = load_model_one("./models/gunshot_sound_model.h5")
# training_sample, sr = librosa.load("./recordings/260600_8.wav")
# training_sample = normalize(training_sample)
# number_of_missing_hertz = 44100 - len(training_sample)
# training_sample = np.array(training_sample.tolist() + [0 for i in range(number_of_missing_hertz)])
# training_sample = training_sample.reshape(-1, 44100, 1)
# probabilities = model.predict(training_sample)
# logger.debug("The model-predicted probability values: " + str(probabilities[0]))
# logger.debug("Model-predicted sample class: " + label_binarizer.inverse_transform(probabilities[:, 0])[0])